# Building a collection

`intake_cesm` support building user-defined collection. This notebook demonstrates how to do this.

In [1]:
import intake_cesm

## Collection definition file

Aspects of the catalog can be defined in the `collection_type_def_file`. This is a yaml file with the following contents.

`collection_columns` : list of columns to include in catalog; for example:
```yaml
collection_columns:
  - resource
  - experiment
  - case
  - component
  ...
```

`replacements` : nested dictionaries of form {column_name: {`to_replace`: `value`}}; this is passed to [`pandas.DataFrame.replace`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html); for example
```yaml
replacements:
    freq:
      monthly: month_1
      daily: day_1
      yearly: year_1
```
(*Note: the current CESM catalog definition does not include the `freq` column.*)

`component_streams` : dictionary with lists of `stream` string for each `component`; for example:
```yaml
component_streams:
    ocn:
      - pop.h.nday1
      - pop.h.nyear1
      - pop.h.ecosys.nday1
      - pop.h.ecosys.nyear1
      - pop.h
    atm:
      - cam.h0
      - cam.h1
      - cam.h2
    ...
```

In [2]:
collection_type_def_file = '../../intake_cesm/cesm_definitions.yml'

## Collection input file

Collections are built from a `yaml` input file containing a nested dictionary. 

It might looks something like this.
```yaml
cesm1_le:  # name of collection
  type: cesm  # type of collection, used to determine build method
  data_sources: # dictionary of data sources to include; each key is an `experiment`
    CTRL: # experiment name followed by list of dictionaries for each ensemble member in the experiment
      - case: b.e11.B1850C5CN.f09_g16.005
        sequence_order: 0
        ensemble: 0
        has_ocean_bgc: True
        year_offset: 1448
        locations:          # ordered list of locations from which to get data
          - name: GLADE
            type: posix
            urlpath: /glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE
          - name: HPSS
            type: hsi
            urlpath: /CCSM/csm/CESM-CAM5-BGC-LE
        component_attrs:
          ocn:
            grid: POP_gx1v6
    20C:
      - case: b.e11.B20TRC5CNBDRD.f09_g16.001
        sequence_order: 0
        ensemble: 1
        has_ocean_bgc: True
        locations:
          - name: GLADE
            type: posix
            urlpath: /glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE
          - name: HPSS
            type: hsi
            urlpath: /CCSM/csm/CESM-CAM5-BGC-LE
        component_attrs:
          ocn:
            grid: POP_gx1v6

      - case: b.e11.B20TRC5CNBDRD.f09_g16.002
        sequence_order: 0
        ensemble: 2
        has_ocean_bgc: True
        locations:
          - name: GLADE
            type: posix
            urlpath: /glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE
          - name: HPSS
            type: hsi
            urlpath: /CCSM/csm/CESM-CAM5-BGC-LE
        component_attrs:
          ocn:
            grid: POP_gx1v6
```

In [3]:
collection_type_def_file = '../../intake_cesm/cesm_definitions.yml'='../../intake_cesm/collection_input_cesm1_le.yml'

## Build the collection

The build method loops over all the experiments and each of the ensemble members therein. It attempts to parse file name; it fails in some instances and skips these files with a warning. If HPSS access is not available (such as from compute nodes on Cheyenne), this resource is omitted from the catalog. 

Ultimately, the collection is saved to disk as a csv file at `collection.active_db`

In [4]:
col = intake_cesm.CESMCollections(collection_input_file=collection_input_file,
                                   collection_type_def_file=collection_type_def_file)
col

INFO:root:Active collection : cesm1_le
INFO:root:Active database: ./collections/cesm1_le.csv
INFO:root:calling build
INFO:root:working on experiment: CTRL
0it [00:00, ?it/s]INFO:root:getting file listing: GLADE:posix:/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE
INFO:root:building file database: GLADE:posix:/glade/collections/cdg/data/cesmLE/CESM-CAM5-BGC-LE

 34%|███▎      | 52759/157253 [00:00<00:01, 89537.81it/s]WARNING:root:Filename: f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.THBOT.200601-201412.nc_temp_.nc does not conform to expected pattern

 39%|███▊      | 60756/157253 [00:00<00:01, 79997.98it/s]WARNING:root:Filename: f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.ERRH2OSNO.200601-201412.nc_temp_.nc does not conform to expected pattern

 53%|█████▎    | 82620/157253 [00:01<00:01, 72260.52it/s]WARNING:root:Filename: f.e11.FAMIPC5CN.f09_f09.rcp85.toga.ens10.clm2.h0.PSNSUN.200601-201412.nc_temp_.nc does not conform to expected pattern

 57%|█████▋    | 89877/157253 

## Examing the collection

`intake_cesm` builds a `pandas.DataFrame` to store the collection. The `DataFrame` is stored as an attribute on the collection object.

In [5]:
col.df

,resource,experiment,case,component,stream,variable,date_range,ensemble,files,files_basename,files_dirname,ctrl_branch_year,year_offset,sequence_order,has_ocean_bgc,grid
0,HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.105,atm,cam.h1,FLNS,20060101-21001231,105,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,b.e11.BRCP85C5CNBDRD.f09_g16.105.cam.h1.FLNS.2...,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,NaN,NaN,1,True,NaN
1,HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.105,atm,cam.h1,FLNSC,20060101-21001231,105,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,b.e11.BRCP85C5CNBDRD.f09_g16.105.cam.h1.FLNSC....,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,NaN,NaN,1,True,NaN
2,HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.105,atm,cam.h1,FLUT,20060101-21001231,105,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,b.e11.BRCP85C5CNBDRD.f09_g16.105.cam.h1.FLUT.2...,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,NaN,NaN,1,True,NaN
3,HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.105,atm,cam.h1,FSNS,20060101-21001231,105,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,b.e11.BRCP85C5CNBDRD.f09_g16.105.cam.h1.FSNS.2...,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,NaN,NaN,1,True,NaN
4,HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.105,atm,cam.h1,FSNSC,20060101-21001231,105,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,b.e11.BRCP85C5CNBDRD.f09_g16.105.cam.h1.FSNSC....,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,NaN,NaN,1,True,NaN
5,HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.105,atm,cam.h1,FSNTOA,20060101-21001231,105,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,b.e11.BRCP85C5CNBDRD.f09_g16.105.cam.h1.FSNTOA...,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,NaN,NaN,1,True,NaN
6,HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.105,atm,cam.h1,ICEFRAC,20060101-21001231,105,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,b.e11.BRCP85C5CNBDRD.f09_g16.105.cam.h1.ICEFRA...,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,NaN,NaN,1,True,NaN
7,HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.105,atm,cam.h1,LHFLX,20060101-21001231,105,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,b.e11.BRCP85C5CNBDRD.f09_g16.105.cam.h1.LHFLX....,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,NaN,NaN,1,True,NaN
8,HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.105,atm,cam.h1,PRECL,20060101-21001231,105,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,b.e11.BRCP85C5CNBDRD.f09_g16.105.cam.h1.PRECL....,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,NaN,NaN,1,True,NaN
9,HPSS:hsi:/CCSM/csm/CESM-CAM5-BGC-LE,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.105,atm,cam.h1,PRECSC,20060101-21001231,105,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,b.e11.BRCP85C5CNBDRD.f09_g16.105.cam.h1.PRECSC...,/CCSM/csm/CESM-CAM5-BGC-LE/atm/proc/tseries/da...,NaN,NaN,1,True,NaN
